In [1]:
%cd ..

/home/romet/projects/ut/milrem/waypoint_planner


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yaml
from model.vae import WaypointVAEGNM
from model.vae import WaypointVAEExperiment
from model.util import load_vae_model
from torchvision import transforms
from data.dataset import MilremDataset, MilremVizDataset
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

import pytorch_lightning as pl
import torch
import torch.nn.functional as F

from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

In [3]:
def plot_data(track, frame_range=None):
    data_root = '/home/romet/data2/datasets/extracted_datasets'
    data = pd.read_csv(f"{data_root}/{track}/csv/extracted_data.csv")
    data.sort_values(by=["timestamp"], inplace=True)
    data.reset_index(inplace=True)
    if frame_range:
        print("Selecting range: ", frame_range)
        data = data.loc[frame_range].copy()
    
    data['velocity'] = np.sqrt(data['gnss_velocity_x']**2 + data['gnss_velocity_y']**2 + data['gnss_velocity_z']**2)
    print("Avg velocity: ", data['velocity'].mean())
    
    old_len = len(data)
    filtered_data = data.dropna(subset=['camera_position_x', 'camera_position_y', 'camera_position_z']).copy()
    print("No camera position filter: ", old_len - len(filtered_data))
    
    filtered_data['diff_x'] = filtered_data['camera_position_x'].diff().abs()
    filtered_data['diff_y'] = filtered_data['camera_position_y'].diff().abs()
    filtered_data['diff_z'] = filtered_data['camera_position_z'].diff().abs()
        
    MAX_TOLERANCE = 1.0
    old_len = len(filtered_data)
    filtered_data = filtered_data[(filtered_data['diff_x'] < MAX_TOLERANCE) & (filtered_data['diff_y'] < MAX_TOLERANCE) & (filtered_data['diff_z'] < MAX_TOLERANCE)]
    print("Position max filter: ", old_len - len(filtered_data))
    
    #MIN_TOLERANCE = 0.0000000000000000001
    #old_len = len(filtered_data)
    #filtered_data = filtered_data[(filtered_data['diff_x'] > MIN_TOLERANCE) | (filtered_data['diff_y'] > MIN_TOLERANCE) | (filtered_data['diff_z'] > MIN_TOLERANCE)]
    #print("Position min filter: ", old_len - len(filtered_data))
    
    VELOCITY_MIN_TOLERANCE = 0.05
    VELOCITY_MAX_TOLERANCE = 2.5
    old_len = len(filtered_data)
    filtered_data = filtered_data[filtered_data['velocity'] > VELOCITY_MIN_TOLERANCE]
    filtered_data = filtered_data[filtered_data['velocity'] < VELOCITY_MAX_TOLERANCE]
    print("Velocity filter: ", old_len - len(filtered_data))
    
    
    removed_ids = list(set(data.index.values) - set(filtered_data.index.values))
    removed_ids.sort()
    removed_rows = data.loc[removed_ids]
    print("Total rows filtered: ", len(removed_rows))
    
    print("Total rows: ", len(data))
    val_start_index = int(0.9*len(data))
    if frame_range and frame_range.start:
        val_start_index = val_start_index + frame_range.start
    #print("Val start index: ", val_start_index)


    model_path = "/home/romet/projects/ut/milrem/drive-any-robot/train/logs/gnm/vae_combined_2023_08_31_19_29_08/latest.pth"
    dataset_path = Path(data_root) / track
    
    action_losses, distance_losses = calculate_losses(model_path, dataset_path, frame_range)


    print("Action loss: ", action_losses.mean())
    print("Distance loss: ", distance_losses.mean())
    
    
    fig, axs = plt.subplots(4, 2, figsize=(12, 12))
    
    marker_size = 3
    removed_marker_size = 10
    axs[0, 0].scatter(filtered_data['camera_position_x'], filtered_data['camera_position_y'], c=filtered_data['camera_position_z']);
    axs[0, 0].set_title("Track")
    
    axs[0, 1].scatter(filtered_data.index.values, filtered_data['camera_position_x'], c='green', s=marker_size)
    axs[0, 1].scatter(removed_rows.index.values, removed_rows['camera_position_x'], c='red', s=removed_marker_size)
    axs[0, 1].set_title("Camera x position")
    
    axs[1, 0].scatter(filtered_data.index.values, filtered_data['camera_position_y'], c='green', s=marker_size)
    axs[1, 0].scatter(removed_rows.index.values, removed_rows['camera_position_y'], c='red', s=removed_marker_size)
    axs[1, 0].set_title("Camera y position")
    
    axs[1, 1].scatter(filtered_data.index.values, filtered_data['camera_position_y'], c='green', s=marker_size)
    axs[1, 1].scatter(removed_rows.index.values, removed_rows['camera_position_y'], c='red', s=removed_marker_size)
    axs[1, 1].set_title("Camera z position")
    
    axs[2, 0].scatter(filtered_data.index.values, filtered_data['velocity'], c='green', s=marker_size)
    axs[2, 0].scatter(removed_rows.index.values, removed_rows['velocity'], c='red', s=removed_marker_size)
    axs[2, 0].set_title("Velocity")
    
    axs[2, 1].plot(filtered_data['velocity'], c='blue')
    axs[2, 1].set_title("Filtered Velocity")

    axs[3, 0].plot(action_losses)
    axs[3, 0].set_title("Action losses")

    axs[3, 1].plot(distance_losses)
    axs[3, 1].set_title("Distance losses")
    
    fig.suptitle(track)
    
    return data

def load_dataset(dataset_path, data_range, config):
    transform = [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
    image_transform = transforms.Compose(transform)
    aspect_ratio = config["image_size"][0] / config["image_size"][1]
    return MilremDataset(dataset_path, image_transform, aspect_ratio, data_range, **config)

def calculate_losses(model_path, dataset_path, data_range=slice(None, None)):
    with open('config/vae.yaml', 'r') as file:
        config = yaml.safe_load(file)

    model = load_vae_model(model_path, config)
    model.eval()

    dataset = load_dataset(dataset_path, data_range, config)
    dataloader = torch.utils.data.DataLoader(dataset, num_workers=1, batch_size=400, shuffle=False, drop_last=True)
    
    pred_actions = []
    pred_distances = []
    
    label_actions = []
    label_distances = []
    
    with torch.no_grad():
        for i, batch in tqdm(enumerate(dataloader)):
            observations, waypoint_img, (action_label, distance_label), metadata = batch
            (distance_pred, action_pred), mu, log_var, z = model(observations, waypoint_img)
            
            pred_actions.extend(action_pred)
            pred_distances.extend(distance_pred)
        
            label_actions.extend(action_label)
            label_distances.extend(distance_label)

    action_losses = np.array([F.mse_loss(x[0], x[1]).item() for x in zip(pred_actions, label_actions)])
    distance_losses = np.array([F.mse_loss(x[0], x[1]).item() for x in zip(pred_distances, label_distances)])
    
    return action_losses, distance_losses

+2023-07-17-13-37-10 (None, 8500) (8700, 11800) (12000, 13900) (14100, 21600) (22500, 25000) (26000, 36000) (37000, 40000) (48000, 54500)
+2023-07-17-14-38-28 (None, 19500)
+2023-07-19-13-12-11 (3500, None)
-2023-07-21-11-52-18 (15000, None) NOK trajectories very wrong
+2023-07-24-13-53-29 (4500, 19000)
2023-07-24-14-25-47 (250, 1250)
+2023-07-24-14-29-06 (4500, 16500) (18500, None)
-2023-07-26-14-22-18 (10000, None) NOK trajectories totally wrong
2023-07-27-14-58-24 (10000, None)
2023-07-27-15-46-09 OK
2023-07-27-16-12-51 OK
2023-08-01-15-47-18 OK
2023-08-02-16-27-51 NOK
-2023-08-04-11-15-22 (25000, None) NOK very high error
2023-08-08-15-40-29 OK
2023-08-08-16-37-28 OK
+2023-08-09-13-44-25 (1500, 20000) (21000, 50500), (51000, None)
2023-08-09-14-07-47 OK
2023-08-10-16-19-31 OK
2023-08-17-16-16-29 OK
2023-08-21-15-17-51 NOK
2023-08-22-15-25-30 NOK
2023-08-23-15-04-12 OK
2023-08-23-15-12-38 OK
2023-08-23-15-17-22 OK
2023-08-23-15-21-21 OK
2023-08-23-15-26-38 OK
2023-08-23-15-57-55 OK
2023-08-24-16-09-18 (12500, 43000)
2023-08-25-15-48-47 OK


In [4]:
#plot_data('2023-07-17-13-37-10', frame_range=slice(26000, 40000));
plot_data('2023-07-17-13-37-10', frame_range=slice(48000, 54500));

Selecting range:  slice(48000, 54500, None)
Avg velocity:  1.2452309735521894
No camera position filter:  0
Position max filter:  1
Velocity filter:  88
Total rows filtered:  89
Total rows:  6501


FileNotFoundError: [Errno 2] No such file or directory: '/home/romet/projects/ut/milrem/drive-any-robot/train/logs/gnm/vae_combined_2023_08_31_19_29_08/latest.pth'